In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, sigmoid_kernel

import numpy as np
import json
import glob

from collections import Counter, defaultdict

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
# nltk.download()
from nltk.corpus import stopwords


In [2]:
f_21 = pd.read_csv('assets/f_21_merge.csv')
w_22 = pd.read_csv('assets/w_22_merge.csv')

In [3]:
def prepare_text(data):

    def lemmatization(texts, allowed_postags=["NOUN", "ADJ"]):
        nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
        texts_out = []
        for text in texts:
            doc = nlp(text)
            new_text = []
            for token in doc:
                if token.pos_ in allowed_postags:
                    new_text.append(token.lemma_)
            texts_out.append(new_text)
            #final = " ".join(new_text)
            #texts_out.append(final)
        return (texts_out)

    lemmatized_texts = lemmatization(data)

    def stop_word_removal(texts):

        flat_texts = [t for text in texts for t in text]
        common_words = [k for k,v in Counter(flat_texts).most_common(10)]
        texts_out = []
        stop = set(stopwords.words('english')+common_words)
        for text in texts:
            new_text = [t for t in text if t.lower() not in stop]
            final = " ".join(new_text)
            texts_out.append(final)
        return (texts_out)

    stop_word_removed_texts = stop_word_removal(lemmatized_texts)


    def gen_words(texts):
        final = []
        for text in texts:
            new = gensim.utils.simple_preprocess(text, deacc=True)
            final.append(new)
        return (final)

    data_words = gen_words(stop_word_removed_texts)
    #data_words = gen_words(lemmatized_texts)


    # Create bigrams and trigrams
    bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
    trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)

    def make_bigrams(texts):
        return([bigram[doc] for doc in texts])

    def make_trigrams(texts):
        return ([trigram[bigram[doc]] for doc in texts])

    data_bigrams = make_bigrams(data_words)
    data_bigrams_trigrams = make_trigrams(data_bigrams)
    
    prepared_text = [" ".join(data) for data in data_bigrams_trigrams]

    return prepared_text

In [4]:
# We should process description first and store it in the dataframe to reduce time

def user_input_rec(df, user_text, num_of_rec = 10):

    #clean df
    df = df.fillna('').drop_duplicates(subset=['course']).reset_index().drop(columns='index')
    df = df.reset_index(drop = True)


    # Text into series
    text_series = pd.Series([user_text])

    # Merge description and text series
    texts = df['description'].append(text_series)

    # Process text
    texts = prepare_text(texts)

    # Vectorize our Text
    count_vect = CountVectorizer()
    cv_mat = count_vect.fit_transform(texts)

    df_cv_words = pd.DataFrame(cv_mat.todense(), columns=count_vect.get_feature_names())

    # Cosine Similarity Matrix
    cosine_sim_mat = cosine_similarity(cv_mat)

    # Get Course ID/Index
    course_indices = pd.Series(df.index, index=df['course'])

    # Input index
    scores = list(enumerate(cosine_sim_mat[-1]))

    # Scores
    # Sort Scores
    sorted_scores = sorted(scores, key=lambda x:x[1], reverse=True)

    # Recommender
    selected_course_indices = [i[0] for i in sorted_scores[1:]]
    selected_course_scores = [i[1] for i in sorted_scores[1:]]

    result = df[df.columns].iloc[selected_course_indices]

    rec_df = pd.DataFrame(result)

    rec_df['similarity_scores'] = selected_course_scores

    return rec_df[:num_of_rec]

In [5]:
user_input_rec(f_21, user_text = 'African american history', num_of_rec = 10)

,Class Nbr,course,Term,Session,Acad Group,Subject,Course Title,description,Component,Time,...,Has WL,Units,sub_title,credits,requirements_distribution,consent,advisory_prerequisites,other_course_info,repeatability,similarity_scores
4514,30612,THTREMUS 440,Fall 2021,Regular Academic Session,"Music, Theatre & Dance",Theatre & Drama (THTREMUS) Open Sections,Topics AfricnAm Thtr,This is a special topics course which addresse...,REC,12-130PM,...,N,3.00,,3,,,THTREMUS 222.,,May not be repeated for credit.,0.632456
35,26151,AAS 558,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",Afroamerican & African Studies (AAS) Open Sect...,Black World Seminar,In this course we will study African American ...,SEM,10-1PM,...,Y,3.00,- Policing Blackness in America,3,,,Graduate standing or permission of instructor.,,May be repeated for a maximum of 6 credit(s).,0.539360
9,24904,AAS 271,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",Afroamerican & African Studies (AAS) Open Sect...,Intr Afr-Am Lit,In this course we will study the emergence and...,LEC,10-1130AM,...,Y,3.00,,3,"HU, RE",With permission of instructor.,AAS 111 (CAAS 111).,,May not be repeated for credit.,0.490290
1600,24908,ENGLISH 274,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",English Language And Literature (ENGLISH) Open...,Intr Afr-Am Lit,In this course we will study the emergence and...,LEC,10-1130AM,...,Y,3.00,,3,"HU, RE",With permission of instructor.,AAS 111 (CAAS 111).,,May not be repeated for credit.,0.490290
1599,30133,ENGLISH 270,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",English Language And Literature (ENGLISH) Open...,Intro to Amer Lit,This class will take a look at how American li...,LEC,10-1130AM,...,Y,3.00,- American Dreams & Realities,3,HU,With permission of instructor.,,,May not be repeated for credit.,0.425628
25,30511,AAS 407,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",Afroamerican & African Studies (AAS) Open Sect...,African Things,"In southeastern Nigeria, an Igbo proverb tells...",SEM,230-4PM,...,Y,3.00,,3,,,HISTART 208/AAS 208.,,May not be repeated for credit.,0.409644
2128,30510,HISTART 406,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",History of Art (HISTART) Open Sections,African Things,"In southeastern Nigeria, an Igbo proverb tells...",SEM,230-4PM,...,Y,3.00,,3,,,HISTART 208/AAS 208.,,May not be repeated for credit.,0.409644
4493,14254,THTREMUS 323,Fall 2021,Regular Academic Session,"Music, Theatre & Dance",Theatre & Drama (THTREMUS) Open Sections,Am Thtr-Drama,"The study of principal American dramatists, an...",REC,1130-1PM,...,Y,3.00,,3,HU,,,,May not be repeated for credit.,0.392232
175,38936,AMCULT 387,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",American Culture (AMCULT) Open Sections,Hist American Jews,Jews today are one of the most visible minorit...,LEC,4-530PM,...,Y,3.00,,3,HU,,,,May not be repeated for credit.,0.357295
1626,31807,ENGLISH 383,Fall 2021,Regular Academic Session,"Literature, Sci, and the Arts",English Language And Literature (ENGLISH) Open...,Jewish Literature,What does an American Jewish novelist write ab...,LEC,1-230PM,...,Y,3.00,- American Jewish Fiction of the 21st Century...,3,HU,,,,May not be repeated for credit.,0.353553
